In [ ]:
%pip install flask transformers torch 

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install sentencepiece --quiet

In [3]:
!python -m pip install --upgrade pip

In [14]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")

# Load model in TensorFlow
model_pt = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

# Example: encode some text and get a prediction
input_text = "Translate English to French: How are you?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model_pt.generate(**inputs)
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Translated text (PyTorch):", translated_text)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Translated text (PyTorch): Comment êtes-vous?


In [1]:
from flask import Flask, request, jsonify
from transformers import pipeline
import torch
import time
import threading

# Check for GPU availability and set device
device = 0 if torch.cuda.is_available() else -1

# Load the translation model (Supports instruction prompts like "translate English to French")
translator = pipeline("text2text-generation", model="google/flan-t5-large", device=device)

# Initialize Flask
app = Flask(__name__)

@app.route('/')
def home():
    return "🟢 Translation API is running!"

@app.route('/translate', methods=['POST'])
def translate_text():
    try:
        # Get request data
        data = request.get_json()

        # Validate input text
        text = data.get('text', '').strip()
        if not text:
            return jsonify({'error': 'Text must be a non-empty string'}), 400

        # Get custom prompt info
        source_lang = data.get('source_lang', 'English').strip()
        target_lang = data.get('target_lang', 'French').strip()

        if not source_lang or not target_lang:
            return jsonify({'error': 'Both source_lang and target_lang must be provided'}), 400

        # Create prompt dynamically
        prompt = f"translate {source_lang} to {target_lang}: {text}"

        # Perform translation
        translated_text = translator(prompt, max_length=256, do_sample=False)

        return jsonify({
            'translation': translated_text[0]['generated_text'],
            'source_lang': source_lang,
            'target_lang': target_lang
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Run Flask app in a separate thread for integration
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Start Flask server in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Wait for server to start
time.sleep(2)  # Allow server time to start


Device set to use cpu


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.17:5000
Press CTRL+C to quit


127.0.0.1 - - [10/Apr/2025 18:57:16] "GET / HTTP/1.1" 200 -
192.168.1.17 - - [10/Apr/2025 18:58:29] "GET / HTTP/1.1" 200 -
192.168.1.17 - - [10/Apr/2025 18:58:29] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
import requests
import json

url = 'http://127.0.0.1:5000/translate'

data = {
    'text': 'Hello, how are you?',
    'source_lang' : 'English',
    'target_lang' : 'French'
}

response = requests.post(url,json=data)

if response.status_code == 200:
    print("Translation:", response.json()['translation'])
else:
    print("Error:", response.json())